In [13]:
import duckdb
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
import threading
from dotenv import load_dotenv
import os

load_dotenv()
thread_local = threading.local()



def get_db_connection():    
    if not hasattr(thread_local, "conn") or thread_local.conn is None:
        for db_path in os.getenv("DB_PATHS").split(":"):
            try:
                conn = duckdb.connect(database=db_path, read_only=False)
                thread_local.conn = conn
                thread_local.db_path = db_path
                print(f" * Database connected: {db_path}")
                break
            except Exception as e:
                print(f" ! Failed to connect to {db_path}: {e}")
                continue
        else:
            raise Exception("Failed to connect to any database path")
    return thread_local.conn

def get_table_name():
    conn = get_db_connection()
    result = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='main';").fetchall()
    table_names = [row[0] for row in result]
    print(f"! Found these tables: {table_names}")
    return table_names

def get_column_names(table_name):
    conn = get_db_connection()
    query = f"SELECT column_name FROM information_schema.columns WHERE table_name='{table_name}';"
    result = conn.execute(query).fetchall()
    column_names = [row[0] for row in result]
    print(f"- Found these columns in {table_name}: {column_names}")
    return column_names

if __name__ == "__main__":
    conn = get_db_connection()
    get_table_name()
    get_column_names("customers")
    get_column_names("geolocation")
    get_column_names("order_items")
    get_column_names("order_payments")
    get_column_names("order_reviews")
    get_column_names("orders")
    get_column_names("product_category_translation")
    get_column_names("products")
    get_column_names("sellers")

 * Database connected: /media/edward/SSD-Data/My Folder/text-to-sql-ai-agent/olist.db
! Found these tables: ['customers', 'geolocation', 'order_items', 'order_payments', 'order_reviews', 'orders', 'products', 'sellers', 'product_category_translation']
- Found these columns in customers: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
- Found these columns in geolocation: ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']
- Found these columns in order_items: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']
- Found these columns in order_payments: ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
- Found these columns in order_reviews: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']
- 